In [ ]:
include("./Bilevel/Bilevel.jl")
include("utils.jl")
using .Bilevel
using Plots
using ArgParse, PrettyTables

In [ ]:
# -- DEFAULT PARAMETERS OF A SIMULATION --
verbose     = true

# ------- Choice of the model -------
bilevel     = false 
storage     = false 

# ------- Profiles parameters -------
nb_days     = 2 
delta_t     = 60 
peak_power  = 7.0 
EV          = false 
EHP         = false 

# ------- Users parameters -------
PV_pen        = 1.0     
storage_pen   = 1.0   
storage_eff   = 0.98
storage_cost  = 500.0   # kEUR/MVA
amort_storage = 15      # years
PV_capa       = 0.4     # max in MVA
PV_cost       = 500.0   # EUR/kWc 
PV_conv_cost  = 200.0   # kEUR/MVA
amort_PV_conv = 10      # years
amort_PV      = 25      # years 
EIC           = 0.3     # EUR/kWh  
EEC           = 0.1     # EUR/kWh   
DSOEC         = 0.1     # EUR/kWh  
GCC           = 80.0    # EUR/kVA_year
cos_phi       = 0.95  

# ------- DSO parameters -------
substation_cost   = 1e3  # kEUR/MVA
amort_DSO         = 50  
interest_rate_DSO = 0.06 
money_basis = 1.0
weight_I = weight_V = 1.0e-2
weight_obj1 = weight_obj2 = 1.0

In [ ]:
include("main_bilevel.jl")
network.buses

In [ ]:
model = Bilevel.build_model(simulation; set_names=true)
simulation.storage

In [ ]:
return_value = Bilevel.solve_model(model, formulation.powerflow)
save_struct(network_topology, "network_topology.json")
save_struct(network, "network_data.json")

In [ ]:
thesis_dir = "/Users/manoncornet/Documents/University/TFE/ThesisWriting/Master_Thesis/figures" 
for t in 1:model[:time_steps]
    Bilevel.print_network_tikz(model[:network_data], t, 40, 40; dir=thesis_dir, filename="OneLevel_graph_$(t)")
end

In [ ]:
Bilevel.print_case_description(model)

In [ ]:
Bilevel.print_network_results(model)

In [ ]:
using JuMP
function print_cost_results(model; latex=false)
    network = model[:network_data]
    DSO_costs = model[:DSO_costs]


    L = get_nb_lines(network)
    K = get_nb_conductors(network)
    lines = network.lines 
    conductors = network.conductors

    # Investements to build new lines in k€
    DSO_fixed_line = value(sum(model[:Alpha][l, k] * lines[l].length * conductors[k].cost for l in 1:L, k in 1:K))

    BASE_POWER = network.pu_basis.base_power
    SUB_COST = DSO_costs.substation
    Ns = get_nb_substations(network)
   
    # Investements to build new substations in k€
    DSO_fixed_sub = value(sum(model[:S_sub_capa][i] * BASE_POWER * SUB_COST for i in 1:Ns))

   
    # Operational costs of losses on the length of the horizon
    DSO_loss = value(model[:DSO_loss_costs]) * DSO_costs.amortization

    # DSO future value: DSO should get back what he must pay per year for the investment + a certain margin
    DSO_future_value = value(model[:DSO_fixed_costs]) * ((1 + DSO_costs.interest_rate)^DSO_costs.amortization) + value(model[:DSO_loss_costs]) * DSO_costs.amortization

    # DSO revenues
    DSO_revenues = value(sum(model[:grid_costs])) * DSO_costs.amortization
    # Other costs
    user_costs = value(sum(model[:user_costs]))
    PV_costs = value(sum(model[:PV_costs]))
    grid_costs = value(sum(model[:grid_costs]))
    energy_costs = value(sum(model[:energy_costs]))
    energy_revenues = value(sum(model[:energy_revenues]))
    if model[:storage]
        stor_costs = value(sum(model[:storage_costs]))
        kpis_header = (["DSO fixed line", "DSO fixed sub", "DSO loss", "DSO future value", "DSO revenues", "User", "PV", "Storage", "Grid connection", "Energy imported", "Energy exported"],
        ["kEUR", "kEUR", "kEUR", "kEUR", "kEUR", "kEUR/year", "kEUR/year", "kEUR/year", "kEUR/year", "kEUR/year", "kEUR/year"])
        kpis = sig_round([DSO_fixed_line DSO_fixed_sub DSO_loss DSO_future_value DSO_revenues user_costs PV_costs stor_costs grid_costs energy_costs energy_revenues])
    end
    kpis_header = (["DSO fixed line", "DSO fixed sub", "DSO loss", "DSO future value", "DSO revenues", "User", "PV", "Grid connection", "Energy imported", "Energy exported"],
        ["kEUR", "kEUR", "kEUR", "kEUR", "kEUR", "kEUR/year", "kEUR/year", "kEUR/year", "kEUR/year", "kEUR/year"])
    kpis = sig_round([DSO_fixed_line DSO_fixed_sub DSO_loss DSO_future_value DSO_revenues user_costs PV_costs grid_costs energy_costs energy_revenues])
    if latex
        pretty_table(kpis, header=kpis_header, backend=Val(:latex))
    else
        pretty_table(kpis, header=kpis_header)
    end
    # returns the table as a Matrix
    return vcat([i[j] for i in kpis_header, j in 1:length(kpis_header[1])], kpis)
end

In [ ]:
function sig_round(x)
    for (i, a) in enumerate(x)
        if isa(a, AbstractFloat)
            x[i] = round(a, sigdigits=4)
            if abs(a) < 1e-3
                x[i] = 0
            end
        end
    end
    return x
end

In [ ]:
print_cost_results(model)

In [ ]:
function print_case_results(model; latex=false)
    DAYS_A_YEAR = 365
    DELTA_T = model[:delta_t]
    NB_PROFILES = model[:nb_sign_days]
   
    MULTIPLIER = DELTA_T/60 * DAYS_A_YEAR / NB_PROFILES
    # PV penetration : p_pv max = peak PV 
    PV_penetration = sum(value.(model[:p_pv_max]))
    # PV energy
    PV_energy = sum(value.(model[:p_pv])) * MULTIPLIER / PV_penetration

    # PV potential
    network = model[:network_data]
    Nu = get_nb_loads(network)
    Ns = get_nb_substations(network)
    T  = model[:time_steps]
    buses = network.buses 
    PV_prod = [(isnothing(buses[Ns + i].PV_installation) ? zeros(Float64, T) : buses[Ns + i].PV_installation.profile.time_serie) for i in 1:Nu]

    PV_potential = sum(PV_prod[i][t] * value(model[:p_pv_max][i]) for i in 1:Nu, t in 1:T) * MULTIPLIER / PV_penetration

    # Energy exchanges with grid
    energy_to_grid = value(sum(model[:p_exp])) * MULTIPLIER
    energy_from_grid = value(sum(model[:p_imp])) * MULTIPLIER

    # Substations capa
    sub_init_capa = [buses[i].S_rating for i in 1:Ns]
    substation1_capacity = sub_init_capa[1]
    substation2_capacity = sub_init_capa[2]
    kpis_header = (["PV penetration", "PV production", "PV potential", "Energy bought from grid", "Energy sold to grid", "Substation1 capacity", "Substation2 capacity"], ["MVA", "MWh/MVA/year", "MWh/MVA/year", "MWh/year", "MWh/year", "MVA", "MVA"])
    kpis = sig_round([PV_penetration PV_energy PV_potential energy_from_grid energy_to_grid substation1_capacity substation2_capacity])
    if latex
        pretty_table(kpis, header=kpis_header, backend=Val(:latex))
    else
        pretty_table(kpis, header=kpis_header)
    end
    # returns the table as a Matrix
    return vcat([i[j] for i in kpis_header, j in 1:length(kpis_header[1])], kpis)
end

In [ ]:
print_case_results(model)

In [ ]:
function print_network_results(model; latex=false)
    network = model[:network_data]
    nb_lines_built = 0
    for l in network.lines 
        if l.built 
            nb_lines_built += 1
        end
    end

    Ns = get_nb_substations(network)
    buses = network.buses
    nb_substations = 0
    for i in 1:Ns
        if buses[i].built
            nb_substations += 1
        end
    end
 
    K = get_nb_conductors(network)
    cond_string = ["Nb lines cond "*string(i) for i in 1:K]

    lines_cond = [[i for (i, a) in enumerate(value.(model[:Alpha])[:, j]) if isapprox(a, 1, rtol=1e-2)] for j in 1:K]

    kpis_header = ([["Nb lines built", "Nb substations"];cond_string], [["-", "-"];fill("-", 1:K)])
    kpis = reshape(string.([[nb_lines_built, nb_substations];lines_cond]),1,:)
    if latex
        pretty_table(kpis, header=kpis_header, backend=Val(:latex))
    else
        pretty_table(kpis, header=kpis_header)
    end
    # returns the table as a Matrix
    return vcat([i[j] for i in kpis_header, j in 1:length(kpis_header[1])], kpis)
end


In [ ]:
print_network_results(model)